In [17]:
!pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/face_alignment-1.3.4-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [1]:
import feedparser

feeds = {
    "vsd_people": "https://vsd.fr/actu-people/feed/",
    "vsd_tv": "https://vsd.fr/tele/feed/",
    "vsd_company": "https://vsd.fr/societe/feed/",
    "vsd_culture": "https://vsd.fr/culture/feed/",
    "vsd_leisure": "https://vsd.fr/loisirs/feed/",
    "public_news": "https://www.public.fr/feed",
    "public_people": "https://www.public.fr/people/feed",
    "public_tv": "https://www.public.fr/tele/feed",
    "public_fashion": "https://www.public.fr/mode/feed",
    "public_royalty": "https://www.public.fr/people/familles-royales/feed"
}

def fetch_articles(feed_url):
    articles = []
    feed = feedparser.parse(feed_url)
    for entry in feed.entries:
        articles.append({
            "title": entry.title,
            "link": entry.link,
            "published": entry.published,
            "summary": entry.summary,
            "content": entry.get("content", [{"value": ""}])[0]["value"]
        })
    return articles

all_articles = []
for category, url in feeds.items():
    articles = fetch_articles(url)
    for article in articles:
        article["category"] = category
    all_articles.extend(articles)

print(f"Fetched {len(all_articles)} articles.")


Fetched 500 articles.


In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for articles
for article in all_articles:
    article['embedding'] = model.encode(article['title'] + " " + article['summary'], convert_to_tensor=True).tolist()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
import pinecone
import os
from dotenv import load_dotenv
from pinecone import ServerlessSpec

load_dotenv()

# Initialize Pinecone
pc = pinecone.Pinecone(api_key=os.getenv("PINECONE_KEY"), environment="us-west1-gcp")

# Define index name and dimension
index_name = "semantic-search"
embedding_dim = 384 


# Check if index exists; if not, create it
if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=embedding_dim, metric="cosine", spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")

index = pc.Index(index_name)

# Add articles to the vector index
for article in all_articles:
    index.upsert([(article["link"], article["embedding"], {
        "title": article["title"],
        "summary": article["summary"],
        "category": article["category"],
        "published": article["published"]
    })])

print("All articles have been upserted successfully.")


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '104f1b12505db38d0f7a7462bcf9173f', 'Date': 'Mon, 13 Jan 2025 11:28:12 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [4]:
import pickle
import os
def load_cached_urls(log_file):
    """Load cached URLs from a pickle file."""
    if os.path.exists(log_file):
        with open(log_file, "rb") as f:
            return pickle.load(f)
    return set()

In [5]:
links = load_cached_urls('cached_urls.pkl')

In [6]:
links

{'https://vsd.fr/70293-je-me-souviens-karin-viard-evoque-sa-rencontre-marquante-avec-lancien-premier-ministre-edouard-philippe/',
 'https://vsd.fr/70325-ca-a-toujours-ete-une-relation-pudique-pierre-garnier-se-livre-sur-ses-liens-avec-son-pere/',
 'https://vsd.fr/70353-philippe-etchebest-sous-le-choc-en-decouvrant-de-la-moisissure-sur-un-objet-servi-aux-clients/',
 'https://vsd.fr/70354-des-rats-valerie-damidot-traumatisee-par-son-passage-a-la-star-academy/',
 'https://vsd.fr/70413-cendrine-dominguez-devoile-le-plus-lourd-secret-de-fort-boyard-jordan-de-luxe-sous-le-choc/',
 'https://vsd.fr/70448-ines-reg-etrille-natasha-st-pier-dans-son-spectacle-cette-parodie-divise-la-toile/',
 'https://vsd.fr/70473-julie-de-loft-story-1-revele-enfin-son-astuce-pour-preserver-son-intimite-avec-son-cheri-christophe/',
 'https://vsd.fr/70512-si-je-meurs-adeline-toniutti-en-larmes-traumatisee-par-son-ex-elle-deballe-tout/',
 'https://vsd.fr/70513-plagiat-menace-le-torchon-brule-entre-francois-valery-et

[{
    "name": "semantic-search",
    "dimension": 384,
    "metric": "cosine",
    "host": "semantic-search-4pzpfwi.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}, {
    "name": "gossip-semantic-search",
    "dimension": 384,
    "metric": "cosine",
    "host": "gossip-semantic-search-4pzpfwi.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}]


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'f6bf58d3ee100c235db92e44896d75a8', 'Date': 'Mon, 13 Jan 2025 11:27:34 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}
